In [1]:
!pip install nx-arangodb
!pip install cugraph-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install nx-cugraph-cu12 --extra-index-url https://pypi.nvidia.com # Requires CUDA-capable GPU
!pip install langchain langchain-community langchain-openai langgraph
!pip install arango-datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4.2
    Uninstalling networkx-3.4.2:
      Successfully uninstalled networkx-3.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 whic

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.4/921.4 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 121.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 186.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/25.0 MB 84.7 MB/s eta 0:00:00
  Attempting uninstall: cuda-python
    Found existing installation: cuda-python 12.8.0
    Uninstalling cuda-python-12.8.0:
      Successfully uninstalled cuda-python-12.8.0
  Attempting uninstall: rmm-cu12
    Found existing installation: rmm-cu12 25.2.0
    Uninstalling rmm-cu12-25.2.0:
      Successfully uninstalled rmm-cu12-25.2.0
  Attempting uninstall: pylibraft-cu12
    Found existing installation: pylibraft-cu12 25.2.0
    Uninstalling pylibraft-cu12-25.2.0:
      Successfully uninstalled pylibraft-cu12-25.2.0
  Attempting uninstall: pylibcugraph-cu12
    Found existing installation: pylibcu

In [10]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install mem0ai streamlit ragas

In [1]:
# 5. Import the required modules
import os
os.environ["NX_CUGRAPH_AUTOCONFIG"]="True"
import networkx as nx
import nx_arangodb as nxadb

from arango import ArangoClient

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import re

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langsmith.wrappers import wrap_openai
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_core.tools import tool
from ragas.llms import LangchainLLMWrapper
from datasets import Dataset
from ragas.metrics import answer_relevancy
from ragas import evaluate
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import faithfulness
from mem0 import MemoryClient
import networkx as nx
import nx_arangodb as nxadb
from nx_arangodb.convert import nxadb_to_nxcg

os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = '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'
os.environ['LANGSMITH_PROJECT'] = 'pr-internal-town-19'

mem0_client = MemoryClient(api_key="m0-deeg0NWOkIEljTTUPQxbJS2ZkFPrzt14WRC85Aax")


[15:39:06 +0000] [INFO]: NetworkX-cuGraph is available.
INFO:nx_arangodb:NetworkX-cuGraph is available.


In [2]:
db = ArangoClient(hosts="").db(username="root", password="", verify=True)

print(db)

<StandardDatabase _system>


In [3]:
G_adb = nxadb.Graph(name="OPEN_INTELLIGENCE", db =db)

G_nxcg = nxadb_to_nxcg(G_adb)

[15:39:15 +0000] [INFO]: Graph 'OPEN_INTELLIGENCE' exists.
INFO:nx_arangodb:Graph 'OPEN_INTELLIGENCE' exists.
[15:39:15 +0000] [INFO]: Default node type set to 'Actor'
INFO:nx_arangodb:Default node type set to 'Actor'
[15:39:32 +0000] [INFO]: Graph 'OPEN_INTELLIGENCE' load took 15.870609998703003s
INFO:nx_arangodb:Graph 'OPEN_INTELLIGENCE' load took 15.870609998703003s
[15:39:32 +0000] [INFO]: NXCG Graph construction took 0.3238213062286377s
INFO:nx_arangodb:NXCG Graph construction took 0.3238213062286377s


In [4]:
arango_graph = ArangoGraph(db)

In [5]:
os.environ["OPENAI_API_KEY"] = ""

llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

In [6]:
aql_query = ""
aql_result = ""

@tool
def text_to_aql_to_text(query: str):
    """This tool is available to invoke the
    ArangoGraphQAChain object, which enables you to
    translate a Natural Language Query into AQL, execute
    the query, and translate the result back into Natural Language. Date in the graph is in ISO format.
    """

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

    chain = ArangoGraphQAChain.from_llm(
    	llm=llm,
    	graph=arango_graph,
    	verbose=True,
      allow_dangerous_requests=True,
     return_aql_query = True,
     return_aql_result = True

    )
    result = chain.invoke(query)

    global aql_query
    global aql_result
    aql_result = result.get("aql_result", "AQL result not found")


    aql_query = result.get("aql_query", "AQL query not found")

    return str(result["result"])

In [7]:
@tool
def text_to_nx_algorithm_to_text(query):
    """This tool invokes a NetworkX Algorithm on the ArangoDB Graph. It accepts a natural language query,
    determines the appropriate algorithm (and metrics) to execute, and translates the results back to natural language.

    If the query (e.g., traversals, shortest path, centrality measures, etc.) can be solved using the Arango Query Language,
    then do not use this tool.
    """
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

    global aql_query
    global aql_result

    aql_query = ""
    aql_result = ""

    ######################
    print("1) Generating initial NetworkX code")
    generated_code = llm.invoke(f"""
I have a NetworkX Graph called `G_nxcg`. It has the following schema: {arango_graph.schema}. You have to generate the best code without any errors.

Important Note:

I already have `db`, `G_nxcg` and networkx (`nx`) defined.

The most important part to remember is to use db.aql.execute for filtering. Only perform filtering where there is a requirement. If an query is to be applied on the complete graph there is no need of running db.aql.execute query. If there is any filtering required then it should be done using db.aql.execute functionality and then you can calculate the metrics like degree centrality, betweeness centrality etc. First perform the filtering and then perform the metrics calculation.

Here is an example of how to write filtering code.

Most important note is that if filtering is required then it should be done using db.aql.execute functionality.

Don't use any for loop for filtering of the nodes.

I have the following graph analysis query: {query}.

Generate the Python Code required to answer the query using the `G_nxcg` object.

In the python code first compute the metrics then filter out the set of nodes which are of our interest.

Be very precise on the NetworkX algorithm you select to answer this query. Think step by step.

Only assume that networkx is installed, and other base python dependencies.

Always set the last variable as `FINAL_RESULT`, which represents the answer to the original query.

Only provide python code that I can directly execute via `exec()`. Do not provide any instructions.

Make sure that `FINAL_RESULT` stores a short & consice answer. Avoid setting this variable to a long sequence.

Your code:
    """).content

    generated_code = re.sub(r"^```python\n|```$", "", generated_code, flags=re.MULTILINE).strip()

    print("Generated Code:")
    print('-'*10)
    print(generated_code)
    print('-'*10)

    ######################
    print("\n2) Pre-execution Reflection: Running multiple iterations")
    
    validated_code = generated_code
    NUM_REFLECTIONS = 1
    for iteration in range(NUM_REFLECTIONS):
        print(f"Pre-execution Reflection iteration {iteration+1}")
        reflection_prompt = f"""
                "You are an expert in network graphs. You have to understand what type of metric is being asked and correct the errors in the code whenever possible."
                "I already have `db`, `G_nxcg` and networkx (`nx`) defined. You have to provide and think which metric would work the best for the user query in the graph answering. Important note is that G_nxcg is already defined."
                "You have to first understand if there is any filtering required or the calculation should be done on full graph. Unless there is any specific node asked by the user, don't do filtering and if filtering is required it should only be done using db.aql.execute"
                "Try to use some parameters of centrality metrics such that it can be executed without errors and is fast enough."
                "You only have to let the issues if there are any and not reproduce the correct code."
I have generated the following Python code for a NetworkX analysis query:
---
{validated_code}
---

The query is:
{query}

Please review the code to ensure it selects the correct metrics based on the query. In particular, if the query involves centrality measures (e.g., degree centrality, betweenness centrality, closeness centrality, eigenvector centrality), verify that the appropriate metric is computed.
If any adjustments are needed to align the code with the query's intent, provide a corrected version of the code. Otherwise, repeat the same code. Whenever you have to select betweenness centrality use K=10.
Only provide the corrected Python code that can be directly executed via exec(), with the final result stored in FINAL_RESULT.
        """
        pre_exec_reflection = llm.invoke(reflection_prompt).content
        pre_exec_reflection = re.sub(r"^```python\n|```$", "", pre_exec_reflection, flags=re.MULTILINE).strip()
        validated_code = pre_exec_reflection
        print("Reflected Code after iteration", iteration+1)
        print('-'*10)
        print(validated_code)
        print('-'*10)

    ######################
    print("\n3) Executing NetworkX code with Reflection on errors")
    global_vars = {"G_nxcg": G_nxcg, "nx": nx, "db": db, "FINAL_RESULT": None}
    attempt = 1
    MAX_ATTEMPTS = 3
    while attempt <= MAX_ATTEMPTS:
        try:
            exec(validated_code, global_vars, global_vars)
            break
        except Exception as e:
            print(f"EXEC ERROR on attempt {attempt}: {e}")
            reflection_prompt = f"""
I already have `db`, `G_nxcg` and networkx (`nx`) defined. I attempted to execute the following Python code:

---
{validated_code}
---

But encountered the following error:
{e}

Please reflect on the error and generate a corrected version of the code that can be executed via exec().
Only provide the corrected Python code that can be directly executed via exec(), with the final result stored in FINAL_RESULT.
            """
            corrected_code = llm.invoke(reflection_prompt).content
            corrected_code = re.sub(r"^```python\n|```$", "", corrected_code, flags=re.MULTILINE).strip()
            print(f"Reflection attempt {attempt} produced corrected code:")
            print('-'*10)
            print(corrected_code)
            print('-'*10)
            validated_code = corrected_code
            attempt += 1

    if attempt > MAX_ATTEMPTS:
        error_message = f"EXEC ERROR after {MAX_ATTEMPTS} attempts: {e}"
        print(error_message)
        return error_message

    FINAL_RESULT = global_vars["FINAL_RESULT"]
    print('-'*10)
    print(f"FINAL_RESULT: {FINAL_RESULT}")
    print('-'*10)

    ######################
    print("4) Formulating final answer")
    nx_to_text = llm.invoke(f"""
I already have `db`, `G_nxcg` and networkx (`nx`) defined with the schema: {arango_graph.schema}

The graph analysis query was:
{query}

I executed the following Python code:
---
{validated_code}
---

The variable FINAL_RESULT is set to: {FINAL_RESULT}

Based on the original query and FINAL_RESULT, generate a short and concise response that answers the query.
Your response:
    """).content

    return nx_to_text



In [8]:
tools = [text_to_aql_to_text, text_to_nx_algorithm_to_text]



def evaluate_aql_results(user_input, final_result, aql_result):
    evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

    sample = SingleTurnSample(
        user_input=user_input,
        response=final_result,
        retrieved_contexts=[str(aql_result)]
    )

    data_samples = {
        'question': [user_input],
        'answer': [final_result],
        'contexts': [[str(aql_result)]],
    }
    dataset = Dataset.from_dict(data_samples)
    score = evaluate(dataset, metrics=[answer_relevancy,faithfulness])
    relevancy_score = score.to_pandas()['answer_relevancy'][0]
    faithfulness_score  = score.to_pandas()['faithfulness'][0]
    final_result += f"\n\n*Faithfulness Score: {round(faithfulness_score,3)}\n\nAnswer Relevancy: {round(relevancy_score,3)}*"

    return final_result

def query_graph(query):
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
    app = create_react_agent(llm, tools)
    final_state = app.invoke({"messages": [{"role": "user", "content": query}]})
    final_result = final_state["messages"][-1].content
    if len(aql_result)>0:
        final_result = evaluate_aql_results(query, final_result, aql_result)
    return final_result


In [30]:
final_result = query_graph("Which is the most influential node in the graph?")
final_result

1) Generating initial NetworkX code
Generated Code:
----------
import networkx as nx

# Compute degree centrality for all nodes
degree_centrality = nx.degree_centrality(G_nxcg)

# Find the node with the highest degree centrality
most_influential_node = max(degree_centrality, key=degree_centrality.get)

# Filter the graph to get the most influential node's details
query = f"""
FOR node IN {most_influential_node}
    RETURN node
"""
filtered_node = db.aql.execute(query)

# Store the result in FINAL_RESULT
FINAL_RESULT = most_influential_node
----------

2) Pre-execution Reflection: Running multiple iterations
Pre-execution Reflection iteration 1
Reflected Code after iteration 1
----------
import networkx as nx

# Compute degree centrality for all nodes
degree_centrality = nx.degree_centrality(G_nxcg)

# Find the node with the highest degree centrality
most_influential_node = max(degree_centrality, key=degree_centrality.get)

# Store the result in FINAL_RESULT
FINAL_RESULT = most_influent

'The most influential node in the graph is "Class/Battles."'

In [18]:
final_result = query_graph("What happened on 25th Feb 2020 in the graph?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Event
FOR event IN Event
    FILTER event.date == "2020-02-25T00:00:00.000Z"
    RETURN event

AQL Result:
[{'_key': 'ALG8333', '_id': 'Event/ALG8333', '_rev': '_jS012ZG--1', 'date': '2020-02-25T00:00:00.000Z', 'dateStamp': 1582588800000, 'description': 'On 25 February 2020, students gathered in Bouira (Bouira) for the 53rd consecutive university hirak protest. [size=no report]', 'fatalities': 0, 'geo': {'type': 'Point', 'coordinates': [36.38, 3.901]}, 'name': 'On 25 February 2020, students ', 'label': 'Protests'}, {'_key': 'ALG8336', '_id': 'Event/ALG8336', '_rev': '_jS012ZG--2', 'date': '2020-02-25T00:00:00.000Z', 'dateStamp': 1582588800000, 'description': 'On 25 February 2020, students gathered in Setif (Setif) for the 53rd consecutive university hirak protest. [size=no report]', 'fatalities': 0, 'geo': {'type': 'Point', 'coordinates': [36.191, 5.409]}, 'name': 'On 25 February 2020, students ', 'label': 'Protests'}, {'

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
final_result = query_graph("Which node in the graph has the highest eigenvector centrality?")
final_result

1) Generating initial NetworkX code
Generated Code:
----------
import networkx as nx

# Compute eigenvector centrality for all nodes in the graph
eigenvector_centrality = nx.eigenvector_centrality(G_nxcg)

# Find the node with the highest eigenvector centrality
highest_eigenvector_node = max(eigenvector_centrality, key=eigenvector_centrality.get)

# Filter the node to get its details if needed (assuming we want to filter based on some criteria)
# Here we assume we want to filter nodes based on a specific property, for example, 'name'
# If no filtering is needed, you can skip this part
filtered_nodes = db.aql.execute("FOR node IN Event FILTER node.name == @name RETURN node", bind_vars={'name': highest_eigenvector_node})

# Get the final result
FINAL_RESULT = highest_eigenvector_node
----------

2) Pre-execution Reflection: Running multiple iterations
Pre-execution Reflection iteration 1
Reflected Code after iteration 1
----------
import networkx as nx

# Compute eigenvector centrality f

In [31]:
final_result = query_graph("Is there any news which happened in the graph on 20th March 2020")
final_result



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Event
FOR e IN Event
    FILTER e.date == "2020-03-20T00:00:00.000Z"
    RETURN e

AQL Result:
[{'_key': 'CAO3110', '_id': 'Event/CAO3110', '_rev': '_jS016Pa-_-', 'date': '2020-03-20T00:00:00.000Z', 'dateStamp': 1584662400000, 'description': 'On 20 March 2020, a group of four unidentified individuals killed an off-duty police officer in Bafoussam town (Mifi, Ouest).', 'fatalities': 1, 'geo': {'type': 'Point', 'coordinates': [5.478, 10.418]}, 'name': 'On 20 March 2020, a group of f', 'label': 'Violence_against_civilians'}, {'_key': 'CAO3098', '_id': 'Event/CAO3098', '_rev': '_jS016Pa-__', 'date': '2020-03-20T00:00:00.000Z', 'dateStamp': 1584662400000, 'description': 'On 20 March 2020, the Ambazonian separatists attacked the village of Vikovi in Bui division (Nord-Ouest) and killed three people who they accused of betraying them.', 'fatalities': 3, 'geo': {'type': 'Point', 'coordinates': [6.2, 10.667]}, 'name': 'On 20 March

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

'On 20th March 2020, several significant events occurred, primarily characterized by violence and protests across various locations:\n\n1. **Bafoussam Town**: Four unidentified individuals killed an off-duty police officer, marking a tragic incident with one fatality. Separately, Ambazonian separatists targeted the village of Vikovi, resulting in the deaths of three individuals accused of betrayal. Military forces raided the village of Okwala, leading to two fatalities and property destruction.\n\n2. **Tuliguuleed**: Clashes erupted when Oromo militants attacked pastoralists, although no fatalities were reported.\n\n3. **Damongo**: A journalist was attacked by two youths who pressured him to stop reporting on local cattle thefts.\n\n4. **Taran Village**: Youths blocked the entrance to the sub-prefecture, demanding the dismissal of the sub-prefect over a disputed constitutional referendum. Unidentified individuals set fire to sub-prefecture offices in Pita subdivision amid rioting again